<a href="https://colab.research.google.com/github/stephfz/py-webscraping101/blob/master/WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AboutMe

**Stephanie Frias**  [@\_stephfz\_](https://twitter.com/_stephfz_)


*   Ingeniera de Software 👩‍💻
*   Sr. SDET @ Evernote 🐘
*   Peruana 🇵🇪




# Web Scraping con python

¿Qué es webscraping?
Es la extracción de data que podemos encontrar en alguna website





## ¿Cómo funciona un web scraper?
Un web scraper toma una url determinada y "se carga" todo el contenido HTML. Es por esto que súper importante entender a full HTML y conocer de css ya que podremos iniciar la extraccion indicandole al scraper en que etiquetas o nodos encontrará la data que le requerimos. 


# ¿Qué vamos a emplear?

In [1]:
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install requests

# Web Scraping de Webs Estáticas 

## "Cargando" la pagina

In [ ]:
import requests
import pprint

URL = 'https://www.buscalibre.cl/libros/search?q=bailarina'
page = requests.get(URL)
pprint.pprint(page.content)

## Extrayendo un _SOLO_ elemento

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')
#buscamos todos los elementos que esten marcados con las clase producto 
results = soup.find_all('div',class_='producto')
print(len(results))

In [ ]:
# tambien podemos todos los resultados
print(soup.prettify())

In [ ]:
print(results[2])

**Obteniendo un valor por etiqueta dentro de un elemento de la sopa**

In [ ]:
element = results[2] 
book_title = element.find('a')['title']
print(book_title)

**Obteniendo el Autor**

In [ ]:
element.find('div',class_='autor').text

**Obteniendo Atributos custom**

In [ ]:
print(element['data-precio'])

**Obteniendo data de un elemento anidado**

In [ ]:
print(element.find_all('div', class_='descuento-v2')[0].text)

In [ ]:
import requests
URL = 'https://www.buscalibre.cl/libros/search?q=bailarina&page=100'
response = requests.get(URL)
not_found = 'Lo sentimos, pero no encontramos lo que buscas'
print(response.status_code)
print (not_found in response.text)

**Juntando todo y a obtener los datos de todos los resultados**



In [ ]:
titles = []
prices = []
discounts = []
authors  = []

not_found_msg = 'Lo sentimos, pero no encontramos lo que buscas'
URL = 'https://www.buscalibre.cl/libros/search?q=bailaira&page='
pag_index = 1

from bs4 import BeautifulSoup


while (True):
  URL = 'https://www.buscalibre.cl/libros/search?q=bailarina&page={0}'.format(pag_index)
  response = requests.get(URL)
  if (not_found_msg not in response.text):
    soup = BeautifulSoup(response.content, 'html.parser')
    #buscamos todos los elementos que esten marcados con las clase producto 
    results = soup.find_all('div',class_='producto')
    for element in results:
      titles.append(element.find('a')['title'])
      authors.append(element.find('div',class_='autor').text)
      prices.append(element['data-precio'])
      discounts.append(element.find_all('div', class_='descuento-v2')[0].text)
    pag_index+= 1
  else:
    break;    

In [ ]:
print(titles)

In [ ]:
import pandas as pd
data = {'title' : titles, 'author':authors, 'price':prices, 'dscto':discounts}
df = pd.DataFrame(data)
df.tail(30)

In [ ]:
# remover el % de la columan dscto
df['dscto'] = df['dscto'].apply(lambda x : x[:x.find('%dcto'):])
df.head()

# Web Scraping con Webs Dinámicas

In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium

In [ ]:

import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException


options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

#datos a ingresar
modelo = "Alfa-Romeo"
annio_fabricacion = "2022-Nuevo"
modelo = "147 1.6 TI"
telefono_cotizante = "997261789"
email_cotizante = "mimail@yahoo.com"
edad_cotizante = "38"
##


driver.get("http://www.segurosimple.com/")

element = driver.find_element("id", "Marca")
element.send_keys(modelo)


element = driver.find_element("id","Año")
element.send_keys(annio_fabricacion)


element = driver.find_element(By.ID,"Modelo")
assert len(element.text) > len(":: Seleccione ::"), \
        "Los modelos de la marca %s no fueron cargados" % modelo
element.send_keys(modelo)

driver.find_element(By.ID,"TelefonoCotizacion").send_keys(telefono_cotizante)

element = driver.find_element(By.ID,"EmailCotizacion")
element.send_keys(email_cotizante)

element = driver.find_element(By.ID,"edadUsuario")
element.send_keys(edad_cotizante)

time.sleep(2)

btn = driver.find_element("id","boton-cotizar")
driver.execute_script("arguments[0].click();",btn)

# Mala practica
#time.sleep(20)
driver.implicitly_wait(20)
try:
    element = driver.find_element(By.ID, "btCotizarOtroVehiculo")
    print(element)
    source = driver.page_sourc
except NoSuchElementException as e:
    print(e)

print(driver.page_source)  # results

In [9]:
from bs4 import BeautifulSoup
page_source = source #driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [ ]:
container = soup.find("div",{"id":'CompadadorResultadoDesktop'}) 
rows = container.findAll('tr')
for row in rows:
  broker_img, broker_price = None, None
  broker_img = (row.find_all('td')[0]).find('img')
  if broker_img != None:
    broker_name = (broker_img['alt'])
    broker_price = (row.find_all('td')[1]).find('h3')
    if broker_price != None:
      broker_price = (broker_price.text)
    print (broker_name,broker_price)    

# Recursos Útiles
* [Guía de inicio de Beautiful Soup](https://j2logo.com/python/web-scraping-con-python-guia-inicio-beautifulsoup/)
* [Selectores CSS](https://flukeout.github.io/)
* [Primeros para pasos con Selenium, ejemplos con python](https://realpython.com/modern-web-automation-with-python-and-selenium)